# Noisy Transmission Simulation
This notebook simulates transmission of a video over a noisy channel model that we propose.

In [12]:
import numpy as np
import cv2
from random import random, randint

In [8]:
!ffmpeg -i test-video.MP4 -c:v libx264 test-video.264 # convert to appropriate codec

ffmpeg version 4.2.3 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 9.3.1 (GCC) 20200523
  configuration: --enable-gpl --enable-version3 --enable-sdl2 --enable-fontconfig --enable-gnutls --enable-iconv --enable-libass --enable-libdav1d --enable-libbluray --enable-libfreetype --enable-libmp3lame --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libopus --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libtheora --enable-libtwolame --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libzimg --enable-lzma --enable-zlib --enable-gmp --enable-libvidstab --enable-libvorbis --enable-libvo-amrwbenc --enable-libmysofa --enable-libspeex --enable-libxvid --enable-libaom --enable-libmfx --enable-amf --enable-ffnvcodec --enable-cuvid --enable-d3d11va --enable-nvenc --enable-nvdec --enable-dxva2 --enable-avisynth --enable-libopenmpt
  libavutil      56. 31.100 / 56. 31.100
  li

In [13]:
from utils import play_video, play_videos
from h264 import NALU, parse_h264

In [9]:
video_path = "test-video.264"
video = parse_h264(video_path)
len(video)

1272

In [14]:
from collections import Counter
Counter([i.type for i in video if isinstance(i,NALU)]) # type 1 are non-IDR frames

Counter({1: 1246, 39: 8, 40: 8, 37: 8, 6: 1})

In [22]:
p = 0.00001 # 0.001%
with open("test.mp4", 'wb') as f:
    N = 0
    for i,nalu in enumerate(video):
        if not isinstance(nalu,bytes):
            N += 1
            f.write(nalu.prefix)
            f.write(nalu.header)
            payload = nalu.payload

            if nalu.type == 1:
                payload = list(payload)
                for i in [randint(0, len(payload)-1) for j in range(len(payload)) if random() <= p]: # indices of bytes to modify
                    n = randint(0,7) # modify a random bit
                    payload[i] ^= 1 << n

                payload = bytes(payload)
            f.write(payload)
        else:
            f.write(nalu)

In [23]:
play_videos(["test.mp4"],N=1000)

[0]

In [21]:
from image_similarity_measures.quality_metrics import ssim,psnr,sre,rmse

In [24]:
def compare_videos(pathI, pathK, measure):
    capI = cv2.VideoCapture(pathI)
    capK = cv2.VideoCapture(pathK)

    measured = []
    
    retI,frameI = capI.read()
    retK,frameK = capK.read()
    while retI:
        retI, fI = capI.read()
        retK, fK = capK.read()
        if not retI:
            break
        if retI: frameI = fI
        if retK: frameK = fK
        measured.append(measure(frameI, frameK))

    capI.release()
    capK.release()
    return measured

In [ ]:
measuredSSIM = compare_videos(video_path, "test.mp4", ssim) # measures difference between frames